In [0]:
import pyspark
import numpy as np
from pyspark.sql import SparkSession
from pyspark.sql.types import StringType, DoubleType, FloatType, IntegerType
from pyspark.sql.functions import pandas_udf, PandasUDFType
import numpy as np
import pandas as pd
ubica_archivo = '/FileStore/tables/sample_data.csv'
df = spark.read.csv(ubica_archivo, inferSchema = True, header = True)
ubica_archivo = '/FileStore/tables/sample_data.csv'
df = spark.read.csv(ubica_archivo, inferSchema = True, header = True)


In [0]:
df.show()

+-------+---+----------+------+-------+
ratings|age|experience|family| mobile|
+-------+---+----------+------+-------+
 3| 32| 9.0| 3| Vivo|
 3| 27| 13.0| 3| Apple|
 4| 22| 2.5| 0|Samsung|
 4| 37| 16.5| 4| Apple|
 5| 27| 9.0| 1| MI|
 4| 27| 9.0| 0| Oppo|
 5| 37| 23.0| 5| Vivo|
 5| 37| 23.0| 5|Samsung|
 3| 22| 2.5| 0| Apple|
 3| 27| 6.0| 0| MI|
 2| 27| 6.0| 2| Oppo|
 5| 27| 6.0| 2|Samsung|
 3| 37| 16.5| 5| Apple|
 5| 27| 6.0| 0| MI|
 4| 22| 6.0| 1| Oppo|
 4| 37| 9.0| 2|Samsung|
 4| 27| 6.0| 1| Apple|
 1| 37| 23.0| 5| MI|
 2| 42| 23.0| 2| Oppo|
 4| 37| 6.0| 0| Vivo|
+-------+---+----------+------+-------+
only showing top 20 rows

In [0]:
def price_range(brand):
    if brand in ['Samsung','Apple']:
        return 'High Price'
    elif brand =='MI':
        return 'Mid Price'
    else:
        return 'Low Price'

In [0]:
# Se crea la función a partir de una función de pandas, se define el tipo de variable de salida
brand_udf=udf(price_range,StringType()) # para dataframes pyspark
df.withColumn('ClasificaPrecio',brand_udf(df['mobile'])).show(10,False)

+-------+---+----------+------+-------+---------------+
ratings|age|experience|family|mobile |ClasificaPrecio|
+-------+---+----------+------+-------+---------------+
3 |32 |9.0 |3 |Vivo |Low Price |
3 |27 |13.0 |3 |Apple |High Price |
4 |22 |2.5 |0 |Samsung|High Price |
4 |37 |16.5 |4 |Apple |High Price |
5 |27 |9.0 |1 |MI |Mid Price |
4 |27 |9.0 |0 |Oppo |Low Price |
5 |37 |23.0 |5 |Vivo |Low Price |
5 |37 |23.0 |5 |Samsung|High Price |
3 |22 |2.5 |0 |Apple |High Price |
3 |27 |6.0 |0 |MI |Mid Price |
+-------+---+----------+------+-------+---------------+
only showing top 10 rows

In [0]:
# También se pueden crear expresiones lambdas

In [0]:
ageRec_udf = udf(lambda age: "young" if age <= 30 else "senior", StringType())
df = df.withColumn("age_group", ageRec_udf(df.age))
df.show(n = 5)

+-------+---+----------+------+-------+---------+
ratings|age|experience|family| mobile|age_group|
+-------+---+----------+------+-------+---------+
 3| 32| 9.0| 3| Vivo| senior|
 3| 27| 13.0| 3| Apple| young|
 4| 22| 2.5| 0|Samsung| young|
 4| 37| 16.5| 4| Apple| senior|
 5| 27| 9.0| 1| MI| young|
+-------+---+----------+------+-------+---------+
only showing top 5 rows

In [0]:
# Los tipos: StringType, IntegerType, DoubleType() 

In [0]:
# Pandas udf: Más rápidas y eficientes comparadas al udf normal. Hay dos clases Scalar GroupedMap
def Faltante100(edad):
    annos_faltantes=(100-edad)
    return(annos_faltantes)  
Faltante100(60)

Out[12]: 40

In [0]:
Faltante100_udf = pandas_udf(Faltante100, IntegerType())
df.withColumn("yrs_left", Faltante100_udf(df['age'])).show(10,False)  

+-------+---+----------+------+-------+---------+--------+
ratings|age|experience|family|mobile |age_group|yrs_left|
+-------+---+----------+------+-------+---------+--------+
3 |32 |9.0 |3 |Vivo |senior |68 |
3 |27 |13.0 |3 |Apple |young |73 |
4 |22 |2.5 |0 |Samsung|young |78 |
4 |37 |16.5 |4 |Apple |senior |63 |
5 |27 |9.0 |1 |MI |young |73 |
4 |27 |9.0 |0 |Oppo |young |73 |
5 |37 |23.0 |5 |Vivo |senior |63 |
5 |37 |23.0 |5 |Samsung|senior |63 |
3 |22 |2.5 |0 |Apple |young |78 |
3 |27 |6.0 |0 |MI |young |73 |
+-------+---+----------+------+-------+---------+--------+
only showing top 10 rows

In [0]:
# Vamos a suponer que se necesita realizar una operación entre varias columnas
# Función diseñara para operar en entradas que son series de pandas (columnas de un pandas dataframe)
def prod(rating,exp):
    x=rating*exp
    return x
prod(5, 3)  

Out[15]: 15

In [0]:
prod_udf = pandas_udf(prod, DoubleType())
df.withColumn("Multiplicacion",prod_udf(df['ratings'],
df['experience'])).show(10,False)

+-------+---+----------+------+-------+---------+--------------+
ratings|age|experience|family|mobile |age_group|Multiplicacion|
+-------+---+----------+------+-------+---------+--------------+
3 |32 |9.0 |3 |Vivo |senior |27.0 |
3 |27 |13.0 |3 |Apple |young |39.0 |
4 |22 |2.5 |0 |Samsung|young |10.0 |
4 |37 |16.5 |4 |Apple |senior |66.0 |
5 |27 |9.0 |1 |MI |young |45.0 |
4 |27 |9.0 |0 |Oppo |young |36.0 |
5 |37 |23.0 |5 |Vivo |senior |115.0 |
5 |37 |23.0 |5 |Samsung|senior |115.0 |
3 |22 |2.5 |0 |Apple |young |7.5 |
3 |27 |6.0 |0 |MI |young |18.0 |
+-------+---+----------+------+-------+---------+--------------+
only showing top 10 rows

In [0]:
@pandas_udf("double", PandasUDFType.GROUPED_AGG)  
# La desviación estándar está dividiendo sobre n y no sobre n-1
def cv(x):
  return np.std(x) / np.mean(x) 

df.groupBy('mobile').agg(cv(df['age'])).show(n = 10)

/databricks/spark/python/pyspark/sql/pandas/functions.py:386: UserWarning: In Python 3.6+ and Spark 3.0+, it is preferred to specify type hints for pandas UDF instead of specifying pandas UDF type which will be deprecated in the future releases. See SPARK-28264 for more details.
 "in the future releases. See SPARK-28264 for more details.", UserWarning)
+-------+-------------------+
 mobile| cv(age)|
+-------+-------------------+
 Apple|0.18981398141672803|
 MI| 0.184402373803228|
 Oppo|0.22473044999997885|
Samsung|0.21753822016127564|
 Vivo|0.05555555555555555|
+-------+-------------------+

In [0]:
# Acá la función cv está definida para la muestra este es el resultado correcto ( acá la desviación estándar se deivide sobre n-1)
@pandas_udf("double", PandasUDFType.GROUPED_AGG)  
# La desviación estándar está dividiendo sobre n y no sobre n-1
def cv(x):
  return x.std() / x.mean() 

df.groupBy('mobile').agg(cv(df['age'])).toPandas()

,mobile,cv(age)
0,Apple,0.205023
1,MI,0.197134
2,Oppo,0.242737
3,Samsung,0.238301
4,Vivo,0.062113


In [0]:
# No funciona
#def cv(x):
#  return np.std(x) / np.mean(x) 

#cv_udf = pandas_udf(cv, DoubleType())
#df.groupBy('mobile').agg(cv_udf(df['age']))


In [0]:
# skew es un método que están en un pandas dataframe
@pandas_udf("double", PandasUDFType.GROUPED_AGG)  
def asimetria(v):
  return v.skew()

consulta = df.groupBy('mobile').agg(asimetria(df['age']))
consulta = consulta.toPandas()
consulta # pandas dataframe, se puede usar matplolib, las librerias usuales de python

,mobile,asimetria(age)
0,Apple,0.029008
1,MI,1.651905
2,Oppo,1.423778
3,Samsung,0.522804
4,Vivo,-2.236068
